# Python: GATE Sensitivity Analysis

In this simple example, we illustrate how the [DoubleML](https://docs.doubleml.org/stable/index.html) package can be used to perfrom a sensitivity analysis for group average treatment effects in the [DoubleMLIRM](https://docs.doubleml.org/stable/guide/models.html#interactive-regression-model-irm) model.


## Data

For illustration purposes, we will work with generated data where the true individual treatment effects are known, to access the performance of the effect estimates.

In [ ]:
import numpy as np
import doubleml as dml

from doubleml.datasets import make_heterogeneous_data
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

We will add an independent feature `V`, which is treated as a confounder in the analysis. This is an simple option to test the robustness of the effect estimates against this "confounder" and evaluate the sensitivity analysis.

In [ ]:
n_obs = 1000
p = 5

np.random.seed(42)
data_dict = make_heterogeneous_data(n_obs, p, binary_treatment=True, n_x=2)
data = data_dict['data']

# add random covariate
data['V'] = np.random.normal(size=(n_obs, 1))
ite = data_dict['effects']

print("Average Treatment Effect: {:.2f}".format(np.mean(ite)))

As mentioned before, the independent feature `'V'`, will be added to the ``DoubleMLData`` object as a possible confounder.

In [ ]:
dml_data = dml.DoubleMLData(data, 'y', 'd')
print(dml_data)

## ATE Estimation and Sensitivity

At first start with the estimation of the ATE and perform a sensitivity analysis. Throughout this example, we will rely on random forest for nuisance estimation. Further, we use $5$ repetitions to increase stability of the results.

In [ ]:
n_rep = 5

ml_g = RandomForestRegressor(n_estimators=100, min_samples_leaf=20, random_state=42)
ml_m = RandomForestClassifier(n_estimators=100, min_samples_leaf=20, random_state=42)

Set the weights to be explicitly `None` and fit the model.

In [ ]:
dml_irm_ate = dml.DoubleMLIRM(
    dml_data,
    ml_g,
    ml_m,
    n_rep=n_rep,
    weights=None)
dml_irm_ate.fit()
print(dml_irm_ate.summary)

The results seem very robust (also because the effect size is very large).

In [ ]:
dml_irm_ate.sensitivity_analysis()
print(dml_irm_ate.sensitivity_summary)

Benchmarking the covariate `'V'`, reveals very low confounding (which is expected, since the covariate `'V'` is independent of the treatment and outcome).

In [ ]:
benchmarking_variable = 'V'
dml_irm_ate.sensitivity_benchmark(benchmarking_set=[benchmarking_variable])

## GATE Estimation and Sensitivity

Next, we will estimate a GATE and perform a sensitivity analysis. Here, we base the group $G$ on feature `'X_0'`

$$1\{X\in G\} = 1\{X_0 \ge 0.5\},$$

as the treatment effect is heterogeneous in this feature.

In [ ]:
group = data['X_0'] >= 0.5

true_group_effect = ite[group].mean()
print("True group effect: {:.2f}".format(true_group_effect))
print("Group vector:\n{}".format(group[:4]))

The correct weights, to identify the GATE, have to still be normalized by the group probabilites, such that

$$\omega(X) = \frac{1\{X_0 \ge 0.5\}}{P(X_0 \ge 0.5)}.$$

Since the weights only depend on the features $X$, we can supply them as a numpy vector.

In [ ]:
weights = group.to_numpy() / group.mean()
print("Weights:\n{}".format(weights[:4]))

In [ ]:
dml_irm_gate = dml.DoubleMLIRM(
    dml_data,
    ml_g,
    ml_m,
    n_rep=n_rep,
    weights=weights)
dml_irm_gate.fit()
print(dml_irm_gate.summary)

Again, we can repeat the sensitivity analysis for the GATE. The results are very similar to the ATE case (with slightly smaller robustness values).

In [ ]:
dml_irm_gate.sensitivity_analysis()
print(dml_irm_gate.sensitivity_summary)

Still, the benchmarking shows little evidence for confounding via the covariate `'V'`.

In [ ]:
benchmarking_variable = 'V'
dml_irm_gate.sensitivity_benchmark(benchmarking_set=[benchmarking_variable])

## GATET Estimation and Sensitivity

Finally, we will estimate a group average treatment effect on the treated (GATET) and perform a sensitivity analysis. Here, we will use the same group as the previous section.

Instead of  considering the effect for all units with
$X\in G$, the GATET only refers the effect for the treated units such that $X\in G$ and $D=1$.

In [ ]:
group = data['X_0'] >= 0.5
group_treated = group & (data['d'] == 1)

true_gatet_effect = ite[group_treated].mean()
print("True GATET effect: {:.2f}".format(true_gatet_effect))
print("Group vector for treated units:\n{}".format(group_treated[:4]))

As for GATEs, the correct weights, to identify the GATET, have to still be normalized by the group probabilites, such that

$$\omega(D,X) = \frac{D\cdot1\{X_0 \ge 0.5\}}{P(D=1, X_0 \ge 0.5)}.$$

As mentioned in the [User Guide](https://docs.doubleml.org/stable/guide/guide.html), the estimation and sensitivity analysis also relies on the conditional expectation of the weights 
$$\bar{\omega}(D,X) = \mathbb{E}[\omega(D,X)|X].$$

Since for GATETs, the weights depend on the treatment $D$, the conditional expectation $\bar{\omega}(D,X)$ differs from $\omega(D, X)$.

Due to the form of the weights, the conditional expectation $\bar{\omega}(D,X)$ reduces to

$$\bar{\omega}(D,X) = \frac{\mathbb{E}[D|X]\cdot 1\{X_0 \ge 0.5\}}{P(D=1, X_0 \ge 0.5)} = \frac{m_0(X)\cdot 1\{X_0 \ge 0.65\}}{P(D=1, X_0 \ge 0.5)},$$
which can be estimated by plugging in the estimated propensity score $\hat{m}(X)$.

All the previous steps of estimation are performed automatically by the `DoubleMLIRM` class if the score is set to `'ATTE'`.

**Remark**: This requires the `weights` argument to be binary and refer to the group indicator $1\{X\in G\} = 1\{X_0 \ge 0.5\}$ not the actual group of treated individuals $1\{D = 1, X\in G\} = D\cdot 1\{X\in G\} $.
Further, the normalization by the group probabilities is then performed automatically by the `DoubleMLIRM` class.

Consequently, we can just set `weights` to the group indicator $1\{X\in G\}$ and fit the model.

In [ ]:
group_indicator = group.to_numpy()
print("Group indicator:\n{}".format(group_indicator[:4]))

In [ ]:
dml_irm_gatet = dml.DoubleMLIRM(
    dml_data,
    ml_g,
    ml_m,
    n_rep=n_rep,
    score='ATTE',
    weights=group_indicator)
dml_irm_gatet.fit()
print(dml_irm_gatet.summary)

In [ ]:
dml_irm_gatet.sensitivity_analysis()
print(dml_irm_gatet.sensitivity_summary)

As for general sensitivity analysis, contour plots can be used to visualize the results.

In [ ]:
dml_irm_gatet.sensitivity_plot()

Again, the benchmarking of "confounding" feature `'V'` shows little evidence for confounding.

In [ ]:
benchmarking_variable = 'V'
dml_irm_gatet.sensitivity_benchmark(benchmarking_set=[benchmarking_variable])